# Cleaning and exploring a collection of tweets
We go to explore the keywords_list collection in MongoDB.
To the current date the collection has been enriched with streamed tweets containing the keyword "tobacco" for some days.

To explore the data set we need:
1. functions to interface with MongoDB
    - query data
    - load data to
    
2. functions to explore the data

3. functions to manipualate the data
    - cleaning 
    - features engineering

We approch the problem by splitting that into 2 sections:
- __Define the functions__
- __Use the functions__

__Note:__ When we query mongo we obtain a cursor object that contain all the documents that matched the query; to work with them we just need to do _FOR document IN cursor_

Since we work with tweet object documents it is important to have always in mind the __anathomy of a tweet__; we use the following online json parser to show our documents in aplain way.

In [1]:
%%HTML
<iframe width="100%" height="1000" src="http://www.jsoneditoronline.org/"></iframe>

-------

# Define the functions

In [2]:
# packagesg

import pymongo
import sys
import json

import nltk
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs

from collections import Counter
from prettytable import PrettyTable

import csv


### load_from_mongo()
INPUT:
- __mongo_db__: database in which we have the cllection we are interested in (string)
- __mongo_db_coll__: collection in which we have the documents we are interested in (string)
- __return_cursor__: if set to TRUE return a cursor object which is a list of the documents that match our query
- __criteria__: it is the query icluded in {} parenthesis an written in javascript syntax (same as Monngo Shell)
- __projection__: it is a second operator to specifythe keys of the document we want to be returned see [db.collection.find()](https://docs.mongodb.com/manual/reference/method/db.collection.find/#find-projection)

OUTPUT:
- __cursor__: by default we obtain a cursor which is a list (iterable object) of the documents matching our query

In [3]:
def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False,
                    criteria=None, projection=None, **mongo_conn_kw):
    
    client = pymongo.MongoClient(**mongo_conn_kw)    
    db = pymongo.database.Database(client, mongo_db)
    coll = db.get_collection(mongo_db_coll)

    if criteria is None:
        criteria = {}
    
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)
    
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]
    

client = pymongo.MongoClient()    
db = pymongo.database.Database(client, 'snowball')
coll = db.get_collection('net')

documents_in_collection = coll.count()

print documents_in_collection

55360


### save_to_mongo()
INPUT:
- __mongo_db__: database in which we have the cllection we are interested in (string)
- __mongo_db_coll__: collection in which we have the documents we are interested in (string)
- __return_cursor__: if set to TRUE return a cursor object which is a list of the documents that match our query
- __criteria__: it is the query icluded in {} parenthesis an written in javascript syntax (same as Monngo Shell)
- __projection__: it is a second operator to specifythe keys of the document we want to be returned see [db.collection.find()](https://docs.mongodb.com/manual/reference/method/db.collection.find/#find-projection)

OUTPUT:
- __cursor__: by default we obtain a cursor which is a list (iterable object) of the documents matching our query

In [3]:
def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    
    db = pymongo.database.Database(client, mongo_db)
     
    coll = db.get_collection(mongo_db_coll)

    return coll.insert(data)

### natural language processing functions
- _processTweet()_: process the tweet text cleaning it; take and return the tweet __text__
- _processWord()_: process a single word; take and return __single word__
- _getStopWordList()_: take a __text file__ containing english stopwords and return a __list of stopwords__
- _getWordsVector()_: teke the __text__ of a tweet preprocess it and return a __list of words__ contained in it

In [4]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet['text'].lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert retweet
    tweet = re.sub('(rt\s)@[^\s]+','RETWEET',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet


def processWord(w):
    #look for 2 or more repetitions of character in a word and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    w = pattern.sub(r"\1\1", w)
    #strip punctuation
    w = w.strip('\'"?,.')
    #check if the word starts with an alphabet
    val = re.search(r"^[a-zA-Z][a-zA-Z0-9-]*$", w)
    if val is None:
        w = 'ABC'
    return w


def getStopWordList(stopWordListFileName):
    st = open(stopWordListFileName, 'r')
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')
    stopWords.append('RETWEET')
    stopWords.append('ABC')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords


def getWordsVector(tweet):
    # initialize vector
    wordsVector = []
    #initialize stopWords
    stopWords = getStopWordList('/Users/nicolavitale/Desktop/twitter_data_analysis/develop/data/stopwords.txt')
    #process tweet and split into words
    tweet = processTweet(tweet)
    words = tweet.split()
    for w in words:
        w = processWord(w)
        if w in stopWords:
            continue
        else:
            wordsVector.append(w.lower())
    return wordsVector

In [5]:
# test
tweet = {"text" : "RT @StepInPuddles: \"See through the trees\" alcohol marker pen and watercolour 8\" X 11\" please retweet if you like https://t.co/tbtrExK9rW"}
print tweet['text']
print "processTweet()"
processed_text = processTweet(tweet)
print processed_text
print "processWord()"
words_list = []
for w in processed_text.split():
    w_processed = processWord(w)
    words_list.append(w_processed)
print words_list
print "getStopWordList()"
stwl = getStopWordList('/Users/nicolavitale/Desktop/twitter_data_analysis/develop/data/stopwords.txt')
print "getWordsVector()"
wv = getWordsVector(tweet)

print wv

RT @StepInPuddles: "See through the trees" alcohol marker pen and watercolour 8" X 11" please retweet if you like https://t.co/tbtrExK9rW
processTweet()
RETWEET "see through the trees" alcohol marker pen and watercolour 8" x 11" please retweet if you like URL
processWord()
['RETWEET', 'see', 'through', 'the', 'trees', 'alcohol', 'marker', 'pen', 'and', 'watercolour', 'ABC', 'x', 'ABC', 'please', 'retweet', 'if', 'you', 'like', 'URL']
getStopWordList()
getWordsVector()
['trees', 'alcohol', 'marker', 'pen', 'watercolour', 'please', 'retweet']


### extract_entities_from_collection()
INPUT:
- __cursor_object__: Python cursor obgect returned as result of a MongoDB query

OUTPUT:
- __result__: returns a dictionary containing the 3 __lists__ of extracted entities, result = {"words": words, "screen_names": screen_names, "hashtags": hashtags}

In [6]:
def extract_entities_from_collection(cursor_object):
    words = []
    screen_names = []
    hashtags = []
    
    result = {"words": words, "screen_names": screen_names, "hashtags": hashtags}

    for tweet in cursor_object:
        
        wordsVector = getWordsVector(tweet)
        for word in wordsVector:
            words.append(word)

        for user_mention in tweet['entities']['user_mentions']: 
            screen_names.append(user_mention['screen_name'])

        for hashtag in tweet['entities']['hashtags']:
            hashtags.append(hashtag['text'].lower().encode("utf-8"))
    
#     result['words'] = Counter(result['words']).most_common()[:10000]
#     result['screen_names'] = Counter(result['screen_names']).most_common()[:10000]
#     result['hashtags'] = Counter(result['hashtags']).most_common()[:10000]
    return(result)

----------

# Use the functions

Today it's July 28 2016; so far we have streamed data with the keyword "tobacco" for 5 days ("keywords_list" in MongoDB).
We are going to explore this collection so far.

### Count occurencies of collection entities: words, hashtags, screen_names

In [7]:
from collections import Counter
from prettytable import PrettyTable
# if 'KeyError: 'text'' is raised execute the following query in mongob
# db.getCollection('top_occurrences0').remove({"text": { $exists: false }})

In [8]:
cur = load_from_mongo('snowball', 'get_hashtags', return_cursor=True)

results = extract_entities_from_collection(cur)


KeyboardInterrupt: 

In [71]:
with open('words_get_words.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['word','count'])
    for row in results['words']:
        csv_out.writerow(row)

with open('words_get_screen_names.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['screen_name','count'])
    for row in results['screen_names']:
        csv_out.writerow(row)

with open('words_get_hashtags.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['hashtag','count'])
    for row in results['hashtags']:
        csv_out.writerow(row)


In [72]:
cur = load_from_mongo('stream', 'get_hashtags_random', return_cursor=True)

results = extract_entities_from_collection(cur)


In [73]:
with open('words_get_words_random.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['word','count'])
    for row in results['words']:
        csv_out.writerow(row)

with open('words_get_screen_names_random.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['screen_name','count'])
    for row in results['screen_names']:
        csv_out.writerow(row)

with open('words_get_hashtags_random.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['hashtag','count'])
    for row in results['hashtags']:
        csv_out.writerow(row)


In [12]:
# ALL THE COLLECTION 'get_hashtags'
cur = load_from_mongo('net', 'tracks', return_cursor=True)

results = extract_entities_from_collection(cur)

for label, data in (('Word', results["words"]), 
                    ('Screen Name', results["screen_names"]), 
                    ('Hashtag', results["hashtags"])):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:50] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print pt

+-----------------+-------+
| Word            | Count |
+-----------------+-------+
| vape            |   508 |
| vaping          |   315 |
| cannabis        |   292 |
| ecig            |   236 |
| marijuana       |   224 |
| vapeporn        |   216 |
| eliquid         |   212 |
| vaporizer       |   198 |
| sorrynotsorry   |   194 |
| ecigs           |   193 |
| vapelife        |   134 |
| rt              |   118 |
| sale            |   103 |
| follow          |    95 |
| enter           |    94 |
| vapefam         |    86 |
| competition     |    83 |
| jax             |    81 |
| storage         |    81 |
| skull           |    81 |
| vapeon          |    72 |
| notblowingsmoke |    64 |
| medical         |    60 |
| aviator         |    60 |
| vapecommunity   |    57 |
| ejuice          |    54 |
| vapelyfe        |    47 |
| via             |    41 |
| free            |    38 |
| smokers         |    37 |
| stop            |    36 |
| dea             |    35 |
| mme             | 

In [18]:
# ALL THE COLLECTION 'get_hashtags_random'
cur = load_from_mongo('stream', 'get_hashtags_random', return_cursor=True)

results = extract_entities_from_collection(cur)

for label, data in (('Word', results["words"]), 
                    ('Screen Name', results["screen_names"]), 
                    ('Hashtag', results["hashtags"])):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:5000] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print pt

+---------------------------------+-------+
| Word                            | Count |
+---------------------------------+-------+
| smoking                         |  3678 |
| smoke                           |  2904 |
| vaping                          |  2385 |
| marijuana                       |  2170 |
| cigarette                       |  2167 |
| drug                            |  2015 |
| ecig                            |  1857 |
| nicotine                        |  1811 |
| tobacco                         |  1771 |
| cig                             |  1554 |
| smoker                          |  1551 |
| vaporizer                       |  1444 |
| shisha                          |  1299 |
| weed                            |  1258 |
| vape                            |  1186 |
| die                             |   961 |
| cute                            |   958 |
| da                              |   845 |
| time                            |   758 |
| cigarettes                    

### Move documents in mongo

In [ ]:
client = pymongo.MongoClient()  
db_from = pymongo.database.Database(client, 'stream')
coll_from = db_from.get_collection('get_hashtags')
# keyword = 'shisha'
cursor_object = coll_from.find({"entities.hashtags" : {"$exists":"true"}, "$where":"this.entities.hashtags.length>0"})

# tweets_from = []

# for tweet_doc in cursor_object:
#     try:
#         for w in tweet_doc['text'].split():
#             if w == keyword:
#                 tweets_from.append(tweet_doc)
#     except:
#         continue
        
# print "Checked for keyword \"%s\"... ... saving possible occurrences." %keyword



for tweet_doc in cursor_object:
    try:
        client = pymongo.MongoClient()        
        db_to = pymongo.database.Database(client, 'snowball')
        coll_to = db_to.get_collection('net')
        
        coll_to.insert_one(tweet_doc)

        print "Saving ... %s \n" % tweet_doc['text']

    except:
        print "Tweet already in collection, continuing."
        continue

print "FINISHED"

Saving ... With all the drug use going on in the olympics,someone check the 3 bags of dope judging the boxing#Rio2016 #peetestforjudges #gotospecsavers 

Saving ... RT @kinni00: Hillary's handler MIA

#TuesdayMotivation  https://t.co/qs0hTOpiPH 

Saving ... RT @BavarianNordic: New drug supply agreement with Bristol-Myers Squibb for CV301 lung #cancer study #NSCLC https://t.co/A5b4w5CQNY 

Saving ... RT @fahrenheitpress: 2/3rds of today's FREE BOOKS have gone to claim yours click on the amazon link - better be quick #FahrenheitFreakOut h… 

Saving ... RT @westcoastereliz: Re polypharmacy #seniors @HealthWorksBC #dementia @SrsAdvocateBC https://t.co/jyfIPyNBuJ 

Saving ... Drug cheats with medals,appalling judging and ticket touting by the suits I've enough of #Rio2016 now #michaelconlon was robbed @cooper_m 

Saving ... #songoftheday

Falling In Reverse - "The Drug In Me Is You" https://t.co/q2Wk7IhH0R 

Saving ... .@mickconlan11 was not beaten! He was robbed by judges who most be inves